# Magic in Numpy
> Matrix Operations, and Linear Algebra by Way of Albrecht Dürer

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]


## Numpy Basics

We will now go over some basic approaches on a seemingly simple matrix for illustrative purposes. Hopefully some of the efficient and useful properties of Numpy become apparent.

A favorite work by a favorite artist, Dürer's *Melencolia I* (1514) includes sophisticated use of mathematical allegory, particularly in the top-right corner.

<img src="melancholia.jpg" width="400"> 



<img src="melancholia_detail.jpg" width="200">


The matrix is thus:


In [52]:
import numpy as np

X = np.array([[16, 3,  2, 13],
              [5, 10, 11, 8],
              [9,  6,  7, 12],
              [4, 15, 14,  1]])
print(X)

[[16  3  2 13]
 [ 5 10 11  8]
 [ 9  6  7 12]
 [ 4 15 14  1]]


In [2]:
type(X)

numpy.ndarray

## Magic Squares

This matrix is purported to be a magic square. We must fit the following constraints:

1) Magic
2) Square

Simple enough. Starting with the second condition, Numpy provides a number of methods. Though magic cubes and tesseracts are possible, we can begin with a square. Here's a simple function to detect if an array is square.

In [3]:
def is_square(M: np.ndarray) -> bool:
    '''
    Arguments: M, a 2-d matrix
    Returns: a boolean, True if square
    '''
    assert M.ndim == 2
    return True if M.shape[0] == M.shape[1] else False

is_square(X)

True

## Vectorized Summation: Magic

Now, the more involved condition. If a square is "magic", the array exhibits the following properties[^1]:

i) Each of the $n$ elements are of the set of distinct positive integers $[1,2,3,...,n^2]$, such that $n$ is the "order" of the square. Dürer thus presents a $4^{th}$ order magic square.

ii) The sum of the $n$ numbers about any horizontal, vertical or main diagnonal is the same number – the "magic" constant. It is known that such magic constants can be given by $ \mathcal{M}(X_n) = \frac{1}{n}\sum_{k=1}^{n^2}k = \frac{1}{2}n(n^2+1)$ .


Aside:
iii) The complement to a magic square is derived from subtracting every number in a given magic square by $n^2 + 1$.

Back to Dürer. We can check each condition as follows.





[^1]: there are others, see https://faculty.etsu.edu/stephen/matrix-magic-squares.pdf)

In [4]:
def is_magic(M, verbose = True)->bool:

    #By constraints i) & ii)
    assert M.shape[0] == M.shape[1], 'Not a square.'

    n = M.shape[0]

    assert np.array_equal(np.sort(M.flatten()), np.arange(n**2) + 1), 'Expected elements from [1,2,...,n^2]'

    column_sums = np.sum(M,axis=0)
    #Note that summing across axis 0 actually returns column-wise sums, and vice-versa.
    row_sums = np.sum(M, axis=1)

    diagonal_sums = np.array([np.trace(M),np.trace(np.fliplr(M))]).astype(int)
    magic_num_sum = np.unique(np.concatenate( (column_sums,row_sums,diagonal_sums) ))
    
    if len(magic_num_sum) == 1:
        if verbose:
            print(f'Magic number is {magic_num_sum} with order {n}.')
        return True

In [5]:
#note:
np.fliplr(X).diagonal().sum() == np.flipud(X).diagonal().sum()

True

In [6]:
#Also note:
np.trace(X) == np.diagonal(X).sum()

True

In [7]:
is_magic(X)

Magic number is [34] with order 4.


True

In [8]:
X

array([[16,  3,  2, 13],
       [ 5, 10, 11,  8],
       [ 9,  6,  7, 12],
       [ 4, 15, 14,  1]])

## Fast Indexing: Gnomon Magic

Dürer's square is actually a Gnomonic Magic Square – that is, each non-overlapping root subsquare bordering the four sides of the square ($2\times 2$ subsquare), as well as the central subsquare, sums to the magic constant of the overarching square.

The Gnomon is the portion of the sundial casting a shadow. In a way we also cast a magic projection on subarrays of our main magic square.

We can verify this easily – in Numpy, arrays can be efficiently split with minimal logic, rather than looping over each element and hard-indexing.


In [9]:
a,b,c,d = [quadrant for sub_x in np.split(X,2, axis = 0) for quadrant in np.split(sub_x,2, axis = 1)]

n = X.shape[0]

n_subsquare = np.sqrt(n).astype(int)
start = n//2 - (n_subsquare // 2)
end = n//2 + (n_subsquare // 2)

e = X[start:end,start:end]

sections = [a, b, c, d, e]
sections

[array([[16,  3],
        [ 5, 10]]),
 array([[ 2, 13],
        [11,  8]]),
 array([[ 9,  6],
        [ 4, 15]]),
 array([[ 7, 12],
        [14,  1]]),
 array([[10, 11],
        [ 6,  7]])]

In [10]:
print(set([sum(s.flatten()) for s in sections]))

{34}


All quadrants sum to the magic number of 34. As such, we have verified the deliberate style of Dürer.

# Linear Algebra

We will now move onto some essential linear algebra operations on the magic square.

In [11]:
X

array([[16,  3,  2, 13],
       [ 5, 10, 11,  8],
       [ 9,  6,  7, 12],
       [ 4, 15, 14,  1]])

## Rank

The rank of a matrix is the number of its linearly independent columns. That is, the dimensionality of the vector space spanned by a matrix's columns (or rows) is given by its rank, such that the span is the smallest linear subspace containing a set of vectors describing the matrix.

We can obtain the span of all linear combinations of some vectors $\vec{u},\vec{v}$ by computing $s\vec{u} + t\vec{v}$ for some scalar constants $s,t$. The dimensionality of the span of the row or column vectors of a matrix thus yields the row or column rank.

We will proceed using Numpy, which proceeds using singular value decompositon (SVD):

In [21]:
rank = np.linalg.matrix_rank(X)
rank

3

Thus we have a rank-deficient matrix, since $3 < 4$. 4 is the column-dimensionality of the Magic Square matrix but the columns only span 3 dimensions. Note that $rank(M) \leq \min (m,n)$ for an $m\times n$ matrix $M$.

Note how Numpy uses the property that the rank is equal to the number of nonzero singular values as follows:

In [25]:
u,s,vh = np.linalg.svd(X)
s

array([3.40000000e+01, 1.78885438e+01, 4.47213595e+00, 6.25921042e-16])

We have 4 nonzero singular values, but the final value is extremely small. Numpy therefore considers this zero as the default tolerance is computed as ` M.max()*max(M.shape)*eps `.

In [29]:
eps = np.finfo(float).eps
tol = X.max()*max(X.shape)*eps
tol

1.4210854715202004e-14

In [36]:
rank == len(s[s>tol])

True

In [37]:
rank

3

## Determinant 

The determinant is a useful encoding of the linear transformation described by a particular $n\times n$ matrix. In geometric terms, it is analagous to the volume scaling factor of the linear transformation described by the matrix.

In other words, this is the volume of the parallelepiped (a rhomboid prism; a cube is to a square as a parallelepiped is to a parallelogram) spanned by the vectors (row or column) of a matrix. The sign of the determinant of a matrix denotes whether or not the orientation of a vector space is preserved by the transformation described by the matrix.

Two simple examples, then Dürer's:

In [49]:
A = np.array([[0,-1],[1,0]])
B = np.array([[-2,0],[0,2]])

$A$ describes a 90-degree counterclockwise (↪️) rotation.

$B$ describes a scaling by a factor of $2$ as well as a reflection about the $y$ axis.

In [50]:
print(np.linalg.det(A), np.linalg.det(B), sep='\n')

1.0
-4.0


A simple rotation does not change "volume" nor orientation. A scaling on $x,y$ and a reflection about the $y$ axis is encoded in the determinant, however: the "volume" is changed in total by a factor of $4$ and the sign is negative, indicating a change in the orientation of previous vector space.

These are simple enough to compute by hand, but even a $4\times 4$ dimensional matrix as provided by Dürer is more onerous. Thankfully, Numpy works well:

In [54]:
X = np.array([[16, 3,  2, 13],
              [5, 10, 11, 8],
              [9,  6,  7, 12],
              [4, 15, 14,  1]])
              
np.linalg.det(X)

1.449507180950607e-12

An interesting observation: Dürer does *not* provide a *pandiagonal magic square*, as the determinant of this order-4 magic square is near to, but $not$, zero. In other words, if the broken diagonals (for instance, $16,11,12,15$, or $3,8,7,4$) summed to the magic number, the determinant would be zero [^1].

## Eigenvectors and Eigenvalues

We solve the characteristic equation of a matrix $M$ to find eigenvalues $\vec{\lambda}$. That is, we solve $|M - \lambda I| = 0$ where $I$ is the identity matrix ($I_{ij} = 1 \ s.t\  i=j, 0\ s.t.\ i\not = j$) of identical dimensionality to $M$.

In the case of Dürer's magic square, we simply subtract a value $\lambda$ from each element on the main diagonal and set the resulting matrix's determinant equal to zero.

We can quickly avoid rote work using Numpy:

In [55]:
eigenvals, eigenvects = np.linalg.eig(X)
eigenvects

array([[ 5.00000000e-01,  8.16496581e-01,  2.23606798e-01,
        -4.08248290e-01],
       [ 5.00000000e-01, -4.08248290e-01, -6.70820393e-01,
         0.00000000e+00],
       [ 5.00000000e-01, -1.76752662e-16,  6.70820393e-01,
        -4.08248290e-01],
       [ 5.00000000e-01, -4.08248290e-01, -2.23606798e-01,
         8.16496581e-01]])

In [61]:
eigenvals

array([ 3.40000000e+01,  8.00000000e+00,  4.84818517e-17, -8.00000000e+00])

Note the interesting property of magic squares: the principal (largest) eigenvalue of a magic square composed of positive
elements **is its magic constant**! Of further note, but not applicable here, is the observation that if a magic square has some negative elements, then its magic
constant is one of its eigenvalues.[^1]

This gives us another way to find the magic number of a magic square.

In [63]:
def get_magic_number(M):
    if is_magic(M) and is_square(M):
        return np.linalg.eig(X)[0].round(1).astype(int)[0]
        
get_magic_number(X)

Magic number is [34] with order 4.


34